In [24]:
import os
import pandas as pd
import nltk
wn = nltk.corpus.wordnet
from digital_manuscript import BnF

In [65]:
term = 'saw'
dog = wn.synsets(term, pos=wn.NOUN)
print(dog)
dog[1].definition()
# hyper = dog[0].hypernyms()
# for x in hyper:
#     print(x, x.hypernyms())

# from nltk.corpus import wordnet as wn
# for ss in wn.synsets(term):
#     print (ss, ss.hypernyms())

[Synset('proverb.n.01'), Synset('saw.n.02'), Synset('power_saw.n.01')]


'hand tool having a toothed blade for cutting'

In [26]:
animal = wn.synset('animal.n.01')
artifact = wn.synset('artifact.n.01')

class r_node:
    def __init__(self, lemma, children=[]):
        self.lemma = lemma
        
#         if lemma == animal:
#             print(children + [lemma,])
        self.children = children
        self.hypernyms = lemma.hypernyms()
        self.parents = [r_node(h, children + [lemma,]) for h in self.hypernyms]
        
    def return_animal_path(self):
        if self.lemma == wn.synset('animal.n.01'):
            return [[self.lemma,],]
        else:
            parent_paths = [n.return_animal_path() for n in self.parents]
            parent_paths = [p for p in parent_paths if len(p) > 0]
            paths = []
            for p in parent_paths:
                paths += p
            for p in paths:
                p.append(self.lemma)
            return paths

    def return_concept_path(self):
        if self.lemma == wn.synset('entity.n.01'):
            return [[self.lemma,],]
        else:
            parent_paths = [n.return_concept_path() for n in self.parents]
            parent_paths = [p for p in parent_paths if len(p) > 0]
            paths = []
            for p in parent_paths:
                paths += p
            for p in paths:
                p.append(self.lemma)
            return paths

In [27]:
dog = wn.synsets('cloth', pos=wn.NOUN)[0]
dn = r_node(dog)
dn.return_concept_path()
# print([p.parents[0].lemma for p in dn.parents])

# animal

[[Synset('entity.n.01'),
  Synset('physical_entity.n.01'),
  Synset('object.n.01'),
  Synset('whole.n.02'),
  Synset('artifact.n.01'),
  Synset('fabric.n.01')]]

In [28]:
squirrel = wn.synsets('squirrel')[0]
sn = r_node(squirrel)
sn.return_animal_path()

[[Synset('animal.n.01'),
  Synset('chordate.n.01'),
  Synset('vertebrate.n.01'),
  Synset('mammal.n.01'),
  Synset('placental.n.01'),
  Synset('rodent.n.01'),
  Synset('squirrel.n.01')]]

In [29]:
fish = wn.synsets('fish', pos=wn.NOUN)[0]
fn = r_node(fish)
fn.return_animal_path()

[[Synset('animal.n.01'),
  Synset('chordate.n.01'),
  Synset('vertebrate.n.01'),
  Synset('aquatic_vertebrate.n.01'),
  Synset('fish.n.01')]]

In [30]:
louse = wn.synsets('louse')[0]
ln = r_node(louse)
ln.return_animal_path()

[[Synset('animal.n.01'),
  Synset('invertebrate.n.01'),
  Synset('arthropod.n.01'),
  Synset('insect.n.01'),
  Synset('louse.n.01')]]

In [31]:
oyster = wn.synsets('oyster')[0]
on = r_node(oyster)
on.return_animal_path()

[[Synset('animal.n.01'),
  Synset('invertebrate.n.01'),
  Synset('mollusk.n.01'),
  Synset('bivalve.n.01'),
  Synset('oyster.n.01')]]

In [32]:
squirrel = wn.synsets('squirrel')[0]
weasel = wn.synsets('weasel')[1] # index 0 is a sneaky person
squirrel.lowest_common_hypernyms(weasel)

snake = wn.synsets('snake')[0]
print(snake.lowest_common_hypernyms(weasel))
print(snake.lowest_common_hypernyms(squirrel))

turtle = wn.synsets('turtle')[1] # index 0 is a turtleneck (lmao)
print(turtle.lowest_common_hypernyms(snake))
print(turtle.lowest_common_hypernyms(squirrel))

fish = wn.synsets('fish', pos=wn.NOUN)[0] # pos = part of speech
swan = wn.synsets('swan')[0]
print(fish.lowest_common_hypernyms(turtle))
print(fish.lowest_common_hypernyms(snake))

louse = wn.synsets('louse')[0]
oyster = wn.synsets('oyster')[0]
print(louse.lowest_common_hypernyms(snake))
print(louse.lowest_common_hypernyms(oyster))

[Synset('vertebrate.n.01')]
[Synset('vertebrate.n.01')]
[Synset('reptile.n.01')]
[Synset('vertebrate.n.01')]
[Synset('vertebrate.n.01')]
[Synset('vertebrate.n.01')]
[Synset('animal.n.01')]
[Synset('invertebrate.n.01')]


In [33]:
path = os.getcwd() + '/../manuscript-object/thesaurus/animal.csv'

df = pd.read_csv(path)
terms = list(set(df['prefLabel_en']))
simple_terms = [t for t in terms if ' ' not in t]
simple_terms


for i in range(len(simple_terms)) :
    if simple_terms[i] in ['weasel','turtle','cuckoo','codfish','mussel','chicken','goldfinch','turtledofe','linnet', 'water-dog']: #first meaning is not the animal. Birds tend to have another bird as first meaning
        if simple_terms[i]=='turtledofe':
            simple_terms[i]='turtledove'
        elif simple_terms[i]=='water-dog':
            simple_terms[i]='water_dog'
        lit = [simple_terms[i],1]
    elif simple_terms[i] in ['ewe','swallow','crayfish','partridge','hart',"calendra",'calandra','hog']:#have to go to third meaning to get the animal or the right bird
        if simple_terms[i] in ["calendra",'calandra']:
            simple_terms[i]='lark'
        lit = [simple_terms[i],2]
    elif simple_terms[i] in ['dragon']:#the mythical creature does not have a link to animal. This is "any of several small tropical Asian lizards capable of gliding by spreading winglike membranes on each side of the body"
        lit = [simple_terms[i],3]
    else :
        lit = [simple_terms[i],0]
    simple_terms[i]=lit

for i in range(len(simple_terms)):
    word=simple_terms[i][0]
    liste = wn.synsets(word,pos=wn.NOUN)
    for item in liste:
        ani = r_node(item)
        if ani.return_animal_path()!=[]:
            simple_terms[i].append(item.definition())
        else : 
            simple_terms[i].append(item.definition() + " (not in tree)")

In [34]:
df = pd.DataFrame(simple_terms)
#df.to_csv('./simple_terms.csv',sep=',',index=False)

In [35]:
class Node(object):

    def __init__(self, value, parent):
        self.value = value
        self.children = []
        if parent is not None:
            parent.children.append(self)

In [36]:
from print_tree import print_tree   #actually using package print-tree2 from pypi

class print_custom_tree(print_tree):

    def get_children(self, node):
        return node.children

    def get_node_str(self, node):
        return str(node.value)

In [37]:
animaln = Node('animal', None)
visited = []
skipped=0
path = os.getcwd() + '/../manuscript-object/simple_terms.csv'
words = pd.read_csv(path)

for animals in words.itertuples() :
    if animals[1] not in ['bombicum','og','verdaule','tellin','petit-gri','aucupio','daot','shell','mutton','barbel','pork']:   #not in wordnet or not in tree
        ani = wn.synsets(animals[1])[animals[2]]
        an = r_node(ani)
        lis = an.return_animal_path()[0]
        for i in range (1,len(lis)):
            name = lis[i].name().split('.')[0]
            if name in ['even-toed_ungulate','odd-toed_ungulate','orthopterous_insect','chordate','placental','ungulate','thrush','oscine','leporid','lagomorph','decapod_crustacean','chordate','diapsid','anapsid','chelonian','bovid','bovine',"anseriform_bird",'wading_bird','columbiform_bird','corvine_bird','cuculiform_bird','aquatic_vertebrate','bony_fish','teleost_fish','gadoid','ganoid','annelid','oligochaete','decapod','soft-finned_fish','phasianid','ambystomid','red_deer','hominid','agamid', 'sporting_dog']: #- make everything go boom also things I don't care about
                skipped+=1
                visited.append(name)
            elif name not in visited :
                name_prec = lis[i-1-skipped].name().split('.')[0]
                skipped=0
                string = name+'n'+' = Node(name,'+ name_prec+'n)'
                exec(string)
                visited.append(name)
            else :
                skipped=0

print_custom_tree(animaln) 


                                  ┌musteline_mammal─weasel
                                  │      ┌wolf
                        ┌carnivore┼canine┤
                        │         │      └dog─hunting_dog─water_dog
                        │         └feline─cat
                        │      ┌squirrel
                        │      ├dormouse
                        ├rodent┼mouse
                        │      ├rat
                        │      └porcupine
                        ├swine─hog
                        │      ┌horse
                        ├equine┤
                        │      └mule
                 ┌mammal┤
                 │      ├primate─homo
                 │      │        ┌sheep─ewe
                 │      │        │      ┌cow
                 │      │        ├cattle┼beef
                 │      │        │      └ox
                 │      ├ruminant┤
                 │      │        ├goat
                 │      │        └deer─hart
                 │      ├bat
     

In [38]:
path = os.getcwd() + '/../manuscript-object/thesaurus/material.csv'

df = pd.read_csv(path)
terms = list(set(df['prefLabel_en']))
terms.pop(0)
simple_terms = [t for t in terms if ' ' not in t]

for i in range(len(simple_terms)):
    if simple_terms[i] in ['earth','soil','lead','filing','spat','ocher','perfume','mold','coral','germ','phlegm','foam','or','ebony','amber','carton','blue','egg']:
        simple_terms[i]=[simple_terms[i],1]
    elif simple_terms[i] in ['pitch','distemper','gold']:
        simple_terms[i]=[simple_terms[i],2]
    elif simple_terms[i] in ['scale']:
        simple_terms[i]=[simple_terms[i],6]
    elif simple_terms[i]=='clofe':
        simple_terms[i]='clove'
        simple_terms[i]=[simple_terms[i],3]
    elif simple_terms[i] in ['black']:
        simple_terms[i]=[simple_terms[i],4]
    else :
        if simple_terms[i] in ['canva','antho','ambergri','verdigri']:
            simple_terms[i]=simple_terms[i]+'s'
        elif simple_terms[i] == 'vernice':
            simple_terms[i]='varnish'
        elif simple_terms[i] == 'salpeter':
            simple_terms[i]='saltpeter'
        elif simple_terms[i] == 'preserf':
            simple_terms[i]='preserves'
        elif simple_terms[i]=='pumouse':
            simple_terms[i]='pumice'
        simple_terms[i]=[simple_terms[i],0]

for i in range(len(simple_terms)):
    word=simple_terms[i][0]
    liste = wn.synsets(word,pos=wn.NOUN)
    for item in liste:
        ani = r_node(item)
        if ani.return_concept_path()!=[]:
            simple_terms[i].append(item.definition())
        else : 
            simple_terms[i].append(item.definition() + " (not an entity)")

In [39]:
df = pd.DataFrame(simple_terms)
#df.to_csv('./Tree/material_terms.csv',sep=',',index=False)

In [40]:
entityn = Node('entity', None)
visited = []
skipped=0

path = os.getcwd() + '/../manuscript-object/Tree/material_terms.csv'
words = pd.read_csv(path)


for material in words.itertuples() :
    if material[1] not in ['☉','☾','☼','☀','glueing','coryal','incarnadine','flin','spalt','grai','aquaforti','colla','vetro', 'goumiche','arene', 'theriac', 'bituman', 'aceto', 'wooden','zedoary','stagno','basan','rowan,','anthos','☿','waxed','toadstone','glair', 'sandiver','fustet','potin','dyeing','taffetum','bullitoyre','ferro','muddying','ferlin','leaded','salted','lucertum','soldered','glued','rooftile','earthen','milla','metalline','stavesacre','bullitoire','couleur','arrabeic','charcoaled','cendrée','cristallin','porfidio','ferri','smoked','maplewood','arène','brouillamini','whitening','real','k','♀','bronzing','thunderstone','litharge','purpurina', 'salty','felin','tinned','prele','crocum','tint','silvered','fresil','verdet','dyed','oiled','horsedung','cocon.','billon','tuf','vermiculari','pulverin','unleaded','solle','felinder','coppera','quarton','batture','eau-de-vie','florey','taffer','wallwort','enameled','ferret-silk','ardide','porkfat','sanguine','magistra','vermeille','cornaline','niello','enamelling','cocon','aspalt','damascening','melli','grè','rocaille','crêpe','aurea','aspalte','solfo','chimolée','enamelled','sap-green','rutum','trwood','stuf','leady','debri','luted','galipot','vermillion','enameling','pearled','persicaire','enilanroc','oiling','otnegra','marcasite','oily','verjuice','gilded','verd','asphaltum','regulu']:   #not in wordnet or not in entity tree
        mater = wn.synsets(material[1])[material[2]]
        mat = r_node(mater)
        lis = mat.return_concept_path()[0]
        for i in range (1,len(lis)):
            name = lis[i].name().split('.')[0]
            if name in ['even-toed_ungulate','angiospermous_tree', 'high-angle_gun','copper-base_alloy','ovum',"dyer's_rocket",'edible_fat','heavier-than-air_craft','reproductive_structure','legging','consumer_goods','ceramic_ware','source_of_illumination','baseball_equipment','hospital_room','sheet_metal','vascular_plant','vertebrate','mammal','placental','plant_product','animal_tissue']: #- make everything go boom also things I don't care about
                skipped+=1
                visited.append(name)
            elif name not in visited :
                name_prec = lis[i-1-skipped].name().split('.')[0]
                skipped=0
                string = name+'n'+' = Node(name,'+ name_prec+'n)'
                exec(string)
                visited.append(name)
            else :
                skipped=0

print_custom_tree(entityn)


                                                      ┌egg
                                                      ├flour
                                            ┌foodstuff┼concoction─dough
                                            │         │                              ┌sugar
                                            │         │                   ┌sweetening┼honey
                                            │         │                   │          └syrup
                                            │         │          ┌flavorer┼spice─clove
                                            │         │          │        └condiment─vinegar
                                            │         ├ingredient┤
                                            │         │          └egg_yolk
                                            │         │             ┌cheese
                                            │         └dairy_product┤
                                            │                  

In [96]:
path = os.getcwd() + '/../manuscript-object/thesaurus/tool.csv'

df = pd.read_csv(path)
terms = list(set(df['prefLabel_en']))
terms.pop(0)
simple_terms = [t for t in terms if ' ' not in t]

for i in range(len(simple_terms)):
    if simple_terms[i] in ['canva']:
        simple_terms[i] = simple_terms[i] +'s'
    elif simple_terms[i] == 'glofe':
        simple_terms[i]='glove'
    elif simple_terms[i] == 'siefe':
        simple_terms[i]='sieve'
    elif simple_terms[i] == 'ditto':
        simple_terms[i] = 'finger'
    elif simple_terms[i] == 'bellow':
        simple_terms[i] = 'bellows'
    if simple_terms[i] in ['brush','table','gimlet','ruler','beater','screw','vat','glove','frail','auger','cast']:
        simple_terms[i]=[simple_terms[i],1]
    elif simple_terms[i] in ['board','saw','rake']:
        simple_terms[i]=[simple_terms[i],2]
    elif simple_terms[i] in ['file','bore']:
        simple_terms[i]=[simple_terms[i],3]
    elif simple_terms[i] in ['stake']:
        simple_terms[i]=[simple_terms[i],4]
    elif simple_terms[i] in ['matrix','bolt']:
        simple_terms[i]=[simple_terms[i],5]
    elif simple_terms[i] in ['press']:
        simple_terms[i]=[simple_terms[i],6]
    else :
        simple_terms[i]=[simple_terms[i],0]

for i in range(len(simple_terms)):
    word=simple_terms[i][0]
    liste = wn.synsets(word,pos=wn.NOUN)
    for item in liste:
        ani = r_node(item)
        if ani.return_concept_path()!=[]:
            simple_terms[i].append(item.definition())
        else : 
            simple_terms[i].append(item.definition() + " (not an entity)")

In [92]:
df = pd.DataFrame(simple_terms)
df.to_csv('./Tree/tool_terms.csv',sep=',',index=False)

In [94]:
entityn = Node('entity', None)
visited = []
skipped=0

path = os.getcwd() + '/../manuscript-object/Tree/tool_terms.csv'
words = pd.read_csv(path)


for tool in words.itertuples() :
    if tool[1] not in ['filed','cannule','hammered','molded','chafing-dish','felin','luted','burnisher','estamiere','pestled','pestling','bullitoire','desgrusouer','clamped','muid','lime-twig','chaple','puncheon','cochiaro','desgrusoue','quarton','cutting-punch','scratch-brush','prele','blast-pipe','esgrusouer','cendrée','fustée','brushed','wirebrush','moulet','sieved','chiseling','fournaise','arene','mattra','cushionet','chasing','fire-steel','grai','well-forged','desramonet','burnished','nailed','semal','stoppered','flin','onglet','fornaise','matrass','chameau','scratch-brushed','underfoot','scissor','trusseaulx','bedsheet','thunderstone','sift','gratteau','ditch-spade','grateau','arson']:   #not in wordnet or not in entity tree
        too = wn.synsets(tool[1])[tool[2]]
        to = r_node(too)
        lis = to.return_concept_path()[0]
        for i in range (1,len(lis)):
            name = lis[i].name().split('.')[0]
            if name in ['even-toed_ungulate','ovum',"dyer's_rocket",'heavier-than-air_craft','reproductive_structure','legging','consumer_goods','ceramic_ware','source_of_illumination','baseball_equipment','hospital_room','sheet_metal','vascular_plant','plant_product','animal_tissue','bone-ash_cup',"plumber's_snake",'high-angle_gun','show-stopper','natural_phenomenon','physical_phenomenon','fruit_tree','external_body_part','kitchen_utensil','cutting_implement','timepiece']: #- make everything go boom also things I don't care about
                skipped+=1
                visited.append(name)
            elif name not in visited :
                name_prec = lis[i-1-skipped].name().split('.')[0]
                skipped=0
                string = name+'n'+' = Node(name,'+ name_prec+'n)'
                exec(string)
                visited.append(name)
            else :
                skipped=0

print_custom_tree(entityn)


                                                      ┌drygoods─white_goods─linen─table_linen─napkin
                                                      │        ┌headdress─hat
                                            ┌commodity┼clothing┤
                                            │         │        └handwear─glove
                                            │         └durables─appliance─home_appliance─kitchen_appliance─oven
                                            │                             ┌ramrod
                                            │                         ┌rod┤
                                            │                         │   └baton
                                            │                         │    ┌rake
                                            │                         │    │                ┌chisel─burin
                                            │                         │    ├cutter─edge_tool┤
                                            

In [98]:
path = os.getcwd() + '/../manuscript-object/thesaurus/body_part.csv'

df = pd.read_csv(path)
terms = list(set(df['prefLabel_en']))
terms.pop(0)
simple_terms = [t for t in terms if ' ' not in t]

for i in range(len(simple_terms)):
    if simple_terms[i] in ['temple','phlegm','teeth']:
        simple_terms[i]=[simple_terms[i],1]
    elif simple_terms[i] in ['matrix']:
        simple_terms[i]=[simple_terms[i],2]
    elif simple_terms[i] in []:
        simple_terms[i]=[simple_terms[i],3]
    elif simple_terms[i] in []:
        simple_terms[i]=[simple_terms[i],4]
    else :
        if simple_terms[i] == 'ditto':
            simple_terms[i]='finger'
        simple_terms[i]=[simple_terms[i],0]

for i in range(len(simple_terms)):
    word=simple_terms[i][0]
    liste = wn.synsets(word,pos=wn.NOUN)
    for item in liste:
        ani = r_node(item)
        if ani.return_concept_path()!=[]:
            simple_terms[i].append(item.definition())
        else : 
            simple_terms[i].append(item.definition() + " (not an entity)")
simple_terms

[['digiti', 0],
 ['saliva',
  0,
  'a clear liquid secreted into the mouth by the salivary glands and mucous glands of the mouth; moistens the mouth and starts the digestion of starches'],
 ['elbow',
  0,
  'hinge joint between the forearm and upper arm and the corresponding joint in the forelimb of a quadruped',
  'a sharp bend in a road or river',
  'a length of pipe with a sharp bend in it',
  'the part of a sleeve that covers the elbow joint',
  'the joint of a mammal or bird that corresponds to the human elbow'],
 ['mouth',
  0,
  'the opening through which food is taken in and vocalizations emerge',
  'the externally visible part of the oral cavity on the face and the system of organs surrounding the opening',
  'an opening that resembles a mouth (as of a cave or a gorge)',
  'the point where a stream issues into a larger body of water',
  'a person conceived as a consumer of food',
  'a spokesperson (as a lawyer)',
  'an impudent or insolent rejoinder',
  'the opening of a jar o

In [99]:
df = pd.DataFrame(simple_terms)
df.to_csv('./Tree/body_part_terms.csv',sep=',',index=False)

In [100]:
entityn = Node('entity', None)
visited = []
skipped=0

path = os.getcwd() + '/../manuscript-object/Tree/body_part_terms.csv'
words = pd.read_csv(path)


for body in words.itertuples() :
    if body[1] not in ['digiti','underfoot','breathe','naribus','manu']:   #not in wordnet or not in entity tree
        bod = wn.synsets(body[1])[body[2]]
        print(body[1],bod)
        bo = r_node(bod)
        lis = bo.return_concept_path()[0]
        for i in range (1,len(lis)):
            name = lis[i].name().split('.')[0]
            if name in ['part','synovial_joint']: #- make everything go boom also things I don't care about
                skipped+=1
                visited.append(name)
            elif name not in visited :
                name_prec = lis[i-1-skipped].name().split('.')[0]
                skipped=0
                string = name+'n'+' = Node(name,'+ name_prec+'n)'
                exec(string)
                visited.append(name)
            else :
                skipped=0

print_custom_tree(entityn)

saliva Synset('saliva.n.01')
elbow Synset('elbow.n.01')
mouth Synset('mouth.n.01')
headache Synset('concern.n.04')
face Synset('face.n.01')
sweating Synset('perspiration.n.02')
teeth Synset('tooth.n.01')
eye Synset('eye.n.01')
nostril Synset('nostril.n.01')
breath Synset('breath.n.01')
stomach Synset('stomach.n.01')
matrix Synset('matrix.n.03')
urine Synset('urine.n.01')
wound Synset('wound.n.01')
nose Synset('nose.n.01')
liver Synset('liver.n.01')
hand Synset('hand.n.01')
fingernail Synset('fingernail.n.01')
finger Synset('finger.n.01')
phlegm Synset('phlegm.n.02')
arm Synset('arm.n.01')
thigh Synset('thigh.n.01')
cheek Synset('cheek.n.01')
ore Synset('ore.n.01')
tongue Synset('tongue.n.01')
nipple Synset('nipple.n.01')
foot Synset('foot.n.01')
knee Synset('knee.n.01')
palm Synset('palm.n.01')
head Synset('head.n.01')
blood Synset('blood.n.01')
leg Synset('leg.n.01')
two Synset('two.n.01')
belly Synset('abdomen.n.01')
hair Synset('hair.n.01')
earwax Synset('cerumen.n.01')
neck Synset(